In [360]:
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
%matplotlib inline

In [4]:
%cd ml-latest-small

[WinError 2] Не удается найти указанный файл: 'ml-latest-small'
d:\Machine_Learning\RS_item_to_item\ml-latest-small


In [5]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [130]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [15]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [16]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [46]:
genres = movies['genres'].apply(lambda x: ' '.join(x.replace(' ', '').replace('-', '').split('|'))).values

In [101]:
genres

array(['Adventure Animation Children Comedy Fantasy',
       'Adventure Children Fantasy', 'Comedy Romance', ..., 'Drama',
       'Action Animation', 'Comedy'], dtype=object)

In [37]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(genres)

In [38]:
neigh = NearestNeighbors(n_neighbors=10, metric='euclidean') 
neigh.fit(X_train_tfidf)

NearestNeighbors(metric='euclidean', n_neighbors=10)

In [71]:
# Посчитаем все уникальные жанры 
genres_ = movies['genres'].apply(lambda x: x.replace(' ', '').replace('-', '').split('|')).values
s = set()
for l in genres_:
    for gen in l:
        s.add(gen)

In [72]:
s

{'(nogenreslisted)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'FilmNoir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'SciFi',
 'Thriller',
 'War',
 'Western'}

In [78]:
test = 'Romance Musical Drama Comedy Fantasy'

In [93]:
X_tfidf = tfidf.transform([test])

res = neigh.kneighbors(X_tfidf, return_distance=True)

In [94]:
res

(array([[0.28950725, 0.28950725, 0.39788764, 0.46180329, 0.5060455 ,
         0.51959334, 0.51959334, 0.53674334, 0.53674334, 0.53674334]]),
 array([[4499, 6527, 5082, 7395,  643, 2236, 4598,  696, 4125, 4278]],
       dtype=int64))

In [95]:
movies.iloc[res[1][0]]

,movieId,title,genres
4499,6660,"Red Shoes, The (1948)",Drama|Fantasy|Musical|Romance
6527,54190,Across the Universe (2007),Drama|Fantasy|Musical|Romance
5082,8008,Brigadoon (1954),Fantasy|Musical|Romance
7395,79702,Scott Pilgrim vs. the World (2010),Action|Comedy|Fantasy|Musical|Romance
643,829,Joe's Apartment (1996),Comedy|Fantasy|Musical
2236,2971,All That Jazz (1979),Drama|Fantasy|Musical
4598,6849,Scrooge (1970),Drama|Fantasy|Musical
696,914,My Fair Lady (1964),Comedy|Drama|Musical|Romance
4125,5927,"Best Little Whorehouse in Texas, The (1982)",Comedy|Drama|Musical|Romance
4278,6245,Sweet Charity (1969),Comedy|Drama|Musical|Romance


## Решил немного повеселиться ) привел теги в форму жанров, так как решил, что основная проблема, почему метод соседей выдавал довольно сомнительный результат, было именно в этом

In [221]:
new_tags = []
for movie in tags['movieId'].unique():
    new_tags.append([movie, ' '.join([tag.replace(' ', '').replace('-', '') for tag in tags[tags['movieId'] == movie]['tag'].unique()])])

In [222]:
new_tags = pd.DataFrame(data=new_tags, columns=['movieId', 'tag'])

In [223]:
movies_with_tags =  movies.merge(new_tags, on='movieId')

In [224]:
movies_with_tags[movies_with_tags['movieId'] == 6273]

,movieId,title,genres,tag
990,6273,In a Lonely Place (1950),Drama|Film-Noir|Mystery|Romance,L.A.


In [225]:
s = set()
for tag in movies_with_tags['tag']:
    s.add(tag)
s

{'crime',
 'Hawkeye JamesFennimoreCooper',
 'claymation creepy dark darkfairytale Surreal TimBurton',
 'SaturdayNightLive',
 'kidnapping remade',
 'InNetflixqueue televangelist',
 'celebrityfetishism mediacentralism systemholism',
 'emmathompson MaggieGyllenhaal modernfantasy romance surreal WillFerrell',
 'Aardman',
 '1900s',
 'SethRogen',
 'atmospheric bleak disturbing gritty harsh',
 'Somethingforeveryoneinthisone...sawitwithoutandplanonseeingitwithkids!',
 'blackhole scifi timetravel ChristopherNolan baddialogue philosophicalissues thoughtprovoking visuallyappealing',
 'WorldWarII',
 'JekyllandHyde',
 'InigoMontoya sixfingeredman',
 'BetteDavis Oscar(BestActress)',
 'HalleBerry HughJackman',
 'virtualreality',
 'drugs lesbian',
 'JohnGrisham',
 'Olympics sports',
 'MarkRuffalo',
 'PearlSBuck',
 'Quakers',
 'Bible',
 'prostitution',
 'survival',
 'freedomofexpression',
 'families',
 'Othello',
 'deafness',
 'atmospheric cinematography depressing funny',
 'basedonatruestory darkcomed

In [226]:
tfidf = TfidfVectorizer()
x_tags_train = tfidf.fit_transform(movies_with_tags['tag'].values)

In [227]:
neigh = NearestNeighbors(n_neighbors=10, metric='euclidean') 
neigh.fit(x_tags_train)

NearestNeighbors(metric='euclidean', n_neighbors=10)

In [228]:
tags_test = 'highschool pixar fun'
x_tags_test = tfidf.transform([tags_test])
res = neigh.kneighbors(x_tags_test, return_distance=True)

In [229]:
res

(array([[0.52905963, 0.87588583, 0.89468927, 0.98973409, 0.98973409,
         0.98973409, 0.98973409, 0.98973409, 0.98973409, 0.98973409]]),
 array([[   0,  544, 1524,  381,  350,  681,  581,  323,  655,  382]],
       dtype=int64))

In [230]:
movies_with_tags.iloc[res[1][0]]

,movieId,title,genres,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar fun
544,2355,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy,Pixar
1524,122918,Guardians of the Galaxy 2 (2017),Action|Adventure|Sci-Fi,fun
381,1380,Grease (1978),Comedy|Musical|Romance,highschool
350,1285,Heathers (1989),Comedy,highschool
681,3210,Fast Times at Ridgemont High (1982),Comedy|Drama|Romance,highschool
581,2581,Never Been Kissed (1999),Comedy|Romance,highschool
323,1246,Dead Poets Society (1989),Drama,highschool HighSchool
655,3071,Stand and Deliver (1988),Comedy|Drama,highschool
382,1381,Grease 2 (1982),Comedy|Musical|Romance,highschool


In [231]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [264]:
user_rate = ratings.groupby('userId').mean()['rating']
film_rate = ratings.groupby('movieId').mean()['rating']

In [306]:
new_rate = ratings[['userId', 'movieId', 'rating']].merge(user_rate, on='userId').merge(film_rate, on='movieId')

In [307]:
new_rate.columns = ['userId', 'movieId', 'rating', 'mean_user_rate', 'mean_film_rate']

In [349]:
x, y = new_rate.drop(['userId', 'movieId', 'rating'], axis=1), new_rate['rating'] * 2


In [354]:
x_train, x_test, y_train, y_test = train_test_split(x, y.astype('int8'), test_size=0.2, random_state=42)

In [356]:
model = RandomForestClassifier()

In [357]:
model.fit(x_train, y_train)

RandomForestClassifier()

In [363]:
model.score(x_test, y_test)

0.31336771122570406

# Не очень понял 2 часть про обучение модели. Какой именно у нас таргет :), я выбрал рейтинг, выставленный конкретным пользователем на конкретный фильм, чтобы как-бы предсказать, что пользователь поставит именно этому фильму. Если я все правильно понял, то с таким кол-вом признаков нормально модель не обучишь. Также не лучше в данном случаее использовать модель классификации, так как рейтинг имеет только 10 уникальных значений 0.5, 1, 1.5 ... ? Прошу поправьте, если я не прав!    
# Заранее спасибо!